# 00.참조

In [ ]:
# How to Build a Backtesting Engine in Python Using Pandas
# https://medium.com/@Jachowskii/how-to-build-a-backtesting-engine-in-python-using-pandas-bc8e532a9e95

In [ ]:
!pip install yfinance

# 01.Import the libraries

In [3]:
import pandas as pd
import numpy as np
import yfinance as yf

# 02.Import stock data

In [65]:
amzn = yf.download("005930.KS","2000-01-01","2020-01-01")

[*********************100%***********************]  1 of 1 completed


In [66]:
amzn

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-04,6000.0,6110.0,5660.0,6110.0,4651.739258,74195000
2000-01-05,5800.0,6060.0,5520.0,5580.0,4248.231934,74680000
2000-01-06,5750.0,5780.0,5580.0,5620.0,4278.686035,54390000
2000-01-07,5560.0,5670.0,5360.0,5540.0,4217.779297,40305000
2000-01-10,5600.0,5770.0,5580.0,5770.0,4392.885254,46880000
...,...,...,...,...,...,...
2019-12-23,56100.0,56400.0,55100.0,55500.0,51506.562500,9839252
2019-12-24,55600.0,55700.0,54800.0,55000.0,51042.535156,11868463
2019-12-26,54700.0,55400.0,54400.0,55400.0,51413.757812,9645034


# 03.Define a trading strategy

In [56]:
def SMA(dataset,array,period):
    """
    이동평균 계산
    @dataset: 주가 데이터, dataframe
    @array: 이동평균을 계산할 칼럼명 목록, list
    @period: 이동평균 기간, int
    @retturn: 이동평균 데이터, dataframe
    @예시
    sma14 = SMA(amzn,["Close"],14)
    sma200 = SMA(amzn,["Close"],200)
    """
    return dataset[cols].rolling(period).mean()

In [57]:
cols = ["Close"]
sma14 = SMA(amzn, cols, 14)
sma200 = SMA(amzn, cols, 200)
sma200

,Close
Date,
2000-01-04,NaN
2000-01-05,NaN
2000-01-06,NaN
2000-01-07,NaN
2000-01-10,NaN
...,...
2019-12-23,47006.25
2019-12-24,47059.00
2019-12-26,47117.00


In [58]:
# entry rules
def crossover(array1,array2):
    """
    @array1: 단기 이동평균
    @array2: 장기 이동평균
    @retturn: 단기, 장기 이평 상향 돌파 여부, dataframe
    @예시
    enter_rules = crossover(sma14, sma200)
    """
    return array1 > array2

In [59]:
enter_rules = crossover(sma14,sma200)

In [60]:
enter_rules

,Close
Date,
2000-01-04,False
2000-01-05,False
2000-01-06,False
2000-01-07,False
2000-01-10,False
...,...
2019-12-23,True
2019-12-24,True
2019-12-26,True


In [61]:
# exit rules
def crossunder(array1,array2):
    """
    @array1: 단기 이동평균, dataframe
    @array2: 장기 이동평균, dataframe
    @return: 단기, 장기 이평 하향 돌파 여부, dataframe    
    @예시
    exit_rules = crossunder(sma14, sma200)
    """
    return array1 < array2

In [62]:
exit_rules = crossunder(sma14,sma200)

In [63]:
enter_rules[enter_rules.index == "2000-10-13"]

,Close
Date,
2000-10-13,False


In [64]:
exit_rules[exit_rules.index == "2000-10-13"]

,Close
Date,
2000-10-13,True


# 04.Define a market position function

In [67]:
# turns on if enter_rules is True and exit_rules is False and
# turns off if exit_rules is True.

In [68]:
def marketposition_generator(dataset,enter_rules,exit_rules):
    """
    @dataset: 데이터프레임
    @enter_rules: 진입 시그널 dataframe series
    @exit_rules: 탈출 시그널 dataframe series

    mp= 1 (on) whenever enter_rules is True and exit_rules is False and
    mp= 0 (off) whenever exit_rules is True.
    """
    dataset["enter_rules"] = enter_rules
    dataset["exit_rules"] = exit_rules

    status = 0
    mp = []
    for (i,j) in zip(enter_rules,exit_rules):
        if status == 0:
            if i == 1 and j != -1:
                status = 1
        else:
            if j == -1:
                status = 0
        mp.append(status)

    dataset["mp"] = mp
    dataset["mp"] = dataset["mp"].shift(1)
    dataset.iloc[0,2] = 0

    return dataset["mp"]

In [29]:
# marketposition_generator(amzn,enter_rules["Close"],exit_rules["Close"])

Date
2000-01-03    NaN
2000-01-04    0.0
2000-01-05    0.0
2000-01-06    0.0
2000-01-07    0.0
             ... 
2019-12-24    1.0
2019-12-26    1.0
2019-12-27    1.0
2019-12-30    1.0
2019-12-31    1.0
Name: mp, Length: 5031, dtype: float64

In [69]:
amzn

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-04,6000.0,6110.0,5660.0,6110.0,4651.739258,74195000
2000-01-05,5800.0,6060.0,5520.0,5580.0,4248.231934,74680000
2000-01-06,5750.0,5780.0,5580.0,5620.0,4278.686035,54390000
2000-01-07,5560.0,5670.0,5360.0,5540.0,4217.779297,40305000
2000-01-10,5600.0,5770.0,5580.0,5770.0,4392.885254,46880000
...,...,...,...,...,...,...
2019-12-23,56100.0,56400.0,55100.0,55500.0,51506.562500,9839252
2019-12-24,55600.0,55700.0,54800.0,55000.0,51042.535156,11868463
2019-12-26,54700.0,55400.0,54400.0,55400.0,51413.757812,9645034


# 05.Define a backtesting function

In [76]:
COSTS = 0.50 # 건별 거래 비용
INSTRUMENT = 1 # 1: 주식, 2: 선물
OPERATION_MONEY = 10_000_000 # 초기 투자금
DIRECTION = "long" # (long or short)
ORDER_TYPE = "market" # (market, limit, stop, etc.)
ENTER_LEVEL = amzn["Open"] # 진입 가격

In [77]:
def apply_trading_system(dataset,direction,order_type,enter_level,enter_rules,exit_rules):
    dataset["enter_rules"] = enter_rules.apply(lambda x: 1 if x == True else 0)
    dataset["exit_rules"] = exit_rules.apply(lambda x: -1 if x == True else 0)
    dataset["mp"] = marketposition_generator(dataset, dataset["enter_rules"],dataset["exit_rules"])

    if order_type == "market":
        # 전일자 시그널이 0에서 당일 1로 변경된 경우
        # 전일자 시초가로 매수
        dataset["entry_price"] = np.where(
            (dataset.mp.shift(1) == 0) & (dataset.mp == 1),
            dataset.Open.shift(1),
            np.nan
        )

        if INSTRUMENT == 1:
            # 매수 수량: 초기 투자금 / 당일 시초가
            dataset["number_of_stocks"] = np.where(
                (dataset.mp.shift(1) == 0) & (dataset.mp == 1),
                OPERATION_MONEY / dataset.Open,
                np.nan
            )
    dataset["entry_price"] = dataset["entry_price"].fillna(method="ffill")

    if INSTRUMENT == 1:
        dataset["number_of_stocks"] = dataset["number_of_stocks"].apply(lambda x: round(x,0)).fillna(method="ffill")

    dataset["events_in"] = np.where((dataset.mp == 1) & (dataset.mp.shift(1) == 0), "entry", "")

    if direction == "long":
        if INSTRUMENT == 1:
            dataset["open_operations"] = (dataset.Close - dataset.entry_price) * dataset.number_of_stocks
            dataset["open_operations"] = np.where(
                (dataset.mp == 1) & (dataset.mp.shift(-1) == 0),
                (dataset.Open.shift(-1) - dataset.entry_price) * dataset.number_of_stocks - 2 * COSTS,
                dataset.open_operations
            )
    else:
        if INSTRUMENT == 1:
            dataset["open_operations"] = (dataset.entry_price - dataset.Close) * dataset.number_of_stocks
            dataset["open_operations"] = np.where(
                (dataset.mp == 1) & (dataset.mp.shift(-1) == 0),
                (dataset.entry_price - dataset.Open.shift(-1)) * dataset.number_of_stocks - 2 * COSTS,
                dataset.open_operations
            )

    dataset["open_operations"] = np.where(dataset.mp == 1, dataset.open_operations, 0)
    dataset["events_out"] = np.where((dataset.mp == 1) & (dataset.exit_rules == -1), "exit", "")
    dataset["operations"] = np.where(
                (dataset.exit_rules == -1) & (dataset.mp == 1),
                dataset.open_operations,
                np.nan
    )
    dataset["closed_equity"] = dataset.operations.fillna(0).cumsum()
    dataset["open_equity"] = dataset.closed_equity + dataset.open_operations - dataset.operations.fillna(0)
    dataset.to_csv("trading_system_export.csv")

    return dataset

In [78]:
sma14 = SMA(amzn, amzn['Close'], 14)    
sma200 = SMA(amzn, amzn['Close'], 200)

enter_rules = crossover(sma14, sma200)
exit_rules = crossunder(sma14, sma200)

trading_system = apply_trading_system(amzn, DIRECTION, ORDER_TYPE, ENTER_LEVEL, enter_rules["Close"], exit_rules["Close"])

In [79]:
net_profit = trading_system["closed_equity"][-1] - OPERATION_MONEY
print(round(net_profit, 2))

5644312.0


In [80]:
amzn

,Open,High,Low,Close,Adj Close,Volume,enter_rules,exit_rules,mp,entry_price,number_of_stocks,events_in,open_operations,events_out,operations,closed_equity,open_equity
Date,,,,,,,,,,,,,,,,,
2000-01-04,6000.0,6110.0,0.0,6110.0,4651.739258,74195000,0,0,NaN,NaN,NaN,,0.0,,NaN,0.0,0.0
2000-01-05,5800.0,6060.0,5520.0,5580.0,4248.231934,74680000,0,0,0.0,NaN,NaN,,0.0,,NaN,0.0,0.0
2000-01-06,5750.0,5780.0,5580.0,5620.0,4278.686035,54390000,0,0,0.0,NaN,NaN,,0.0,,NaN,0.0,0.0
2000-01-07,5560.0,5670.0,5360.0,5540.0,4217.779297,40305000,0,0,0.0,NaN,NaN,,0.0,,NaN,0.0,0.0
2000-01-10,5600.0,5770.0,5580.0,5770.0,4392.885254,46880000,0,0,0.0,NaN,NaN,,0.0,,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-23,56100.0,56400.0,55100.0,55500.0,51506.562500,9839252,1,0,1.0,44800.0,215.0,,2300500.0,,NaN,15644312.0,17944812.0
2019-12-24,55600.0,55700.0,54800.0,55000.0,51042.535156,11868463,1,0,1.0,44800.0,215.0,,2193000.0,,NaN,15644312.0,17837312.0
2019-12-26,54700.0,55400.0,54400.0,55400.0,51413.757812,9645034,1,0,1.0,44800.0,215.0,,2279000.0,,NaN,15644312.0,17923312.0
